# Data Visualization of Bird Strikes between 2000 – 2011

In [2]:
!pip install --upgrade openpyxl



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: D:\New folder\python.exe -m pip install --upgrade pip


In [2]:


import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

df = pd.read_excel('Bird Strikes Data.xlsx')


df['FlightDate'] = pd.to_datetime(df['FlightDate'], errors='coerce')  
df['Year'] = df['FlightDate'].dt.year  


required_columns = [
    'Year', 'Aircraft: Airline/Operator', 'Airport: Name', 
    'When: Phase of flight', 'Altitude bin', 'Feet above ground'
]

for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Missing required column: {col}")


yearly_bird_strikes = df.groupby('Year').size().reset_index(name='Bird Strikes')
fig1 = px.bar(yearly_bird_strikes, 
              x='Year', y='Bird Strikes', 
              title='Yearly Bird Strikes',
              color='Bird Strikes',  
              color_continuous_scale=px.colors.sequential.Viridis,  
              labels={'Bird Strikes': 'Number of Bird Strikes'},
              template='plotly_dark')



# Top 10 Airlines Bird Strikes

top_airlines = df['Aircraft: Airline/Operator'].value_counts().nlargest(10).reset_index()
top_airlines.columns = ['Airline', 'Bird Strikes']
fig2 = px.bar(top_airlines, x='Airline', y='Bird Strikes', 
              title='Top 10 Airlines with Bird Strikes',
              color_discrete_sequence=['#ff7f0e'],  # Orange color
              labels={'Airline': 'Airline', 'Bird Strikes': 'Bird Strikes'},
              template='plotly_dark')



# Top 50 Airports Bird Strikes


top_airports = df['Airport: Name'].value_counts().nlargest(50).reset_index()
top_airports.columns = ['Airport', 'Bird Strikes']
fig3 = px.bar(top_airports, x='Airport', y='Bird Strikes', 
              title='Top 50 Airports with Bird Strikes',
              color_discrete_sequence=['#2ca02c'],  # Green color
              labels={'Airport': 'Airport', 'Bird Strikes': 'Bird Strikes'},
              template='plotly_dark')




# Bird Strikes by Phase of Flight


fig4 = px.pie(df, names='When: Phase of flight', title='Bird Strikes by Phase of Flight',
              color_discrete_sequence=px.colors.sequential.Plasma,  # Use a sequential color scale
              template='plotly_dark', hole=0.3)



# Adjust pie chart size by manually setting its layout


fig4.update_layout(
    autosize=False,
    width=600,  # Adjust width for pie chart
    height=600,  # Adjust height for pie chart
)




# Altitude of Bird Strikes



fig5 = px.histogram(df, x='Altitude bin', title='Altitude of Bird Strikes',
                    color_discrete_sequence=['#d62728'],  # Red color
                    template='plotly_dark', color='Altitude bin', nbins=20)




# Average Altitude by Flight Phase



avg_altitude = df.groupby('When: Phase of flight')['Feet above ground'].mean().reset_index()
fig6 = px.bar(avg_altitude, x='When: Phase of flight', y='Feet above ground', 
              title='Average Altitude of Bird Strikes by Flight Phase',
              color_discrete_sequence=['#9467bd'],  # Purple color
              labels={'When: Phase of flight': 'Phase of Flight', 'Feet above ground': 'Average Altitude (Feet)'},
              template='plotly_dark')



# Combine Plots into a Dashboard (Interactive Subplots)




specs = [[{}, {}],
         [{}, {'type': 'domain'}],  
         [{}, {}]]

fig = make_subplots(rows=3, cols=2, 
                    subplot_titles=("Yearly Bird Strikes", "Top 10 Airlines", "Top 50 Airports", 
                                    "Bird Strikes by Phase", "Altitude of Bird Strikes", 
                                    "Average Altitude of Flight Phase"),
                    specs=specs,
                    vertical_spacing=0.25,  
                    horizontal_spacing=0.2)  


fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)
fig.add_trace(fig3.data[0], row=2, col=1)
fig.add_trace(fig4.data[0], row=2, col=2)  
fig.add_trace(fig5.data[0], row=3, col=1)
fig.add_trace(fig6.data[0], row=3, col=2)


fig.update_layout(
    height=1400,  
    width=1600,  
    title_text="Bird Strikes Interactive Dashboard",  
    title_x=0.5, 
    title_font=dict(size=24),  
    title_pad=dict(t=50),  
    showlegend=False,  
    margin=dict(t=100, b=50, l=50, r=50),  
    template="plotly_dark",
    font=dict(size=12),  
)


fig.update_annotations(font_size=16)  


fig.show()
